In [1]:
import os
from pathlib import Path
import sys
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import bootstrap
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from fairlearn.reductions import ExponentiatedGradient, DemographicParity

In [2]:
df_train = pl.read_parquet("C:\\Users\\User\\Downloads\\train_features.parquet")
df_test = pl.read_parquet("C:\\Users\\User\\Downloads\\test_features.parquet")

In [3]:
df_train.head()

query_id,report_date,target,rn,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11
u32,datetime[ns],bool,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,2024-01-11 00:00:00,false,4,0.693,0.1,0.245833,3.376496,0.7777,0.0,2.87166,0.97,1.0,4.6,-0.196707
2,2024-01-11 00:00:00,false,5,0.281,0.1,0.108333,2.909165,0.7882,0.0,1.978092,0.96,0.89,4.8,0.034091
2,2024-01-11 00:00:00,false,8,0.319,0.183333,0.154167,1.725731,0.688,0.5,2.083885,0.95,0.9,4.7,0.286658
2,2024-01-11 00:00:00,false,10,0.281,0.1,0.108333,3.759698,0.7882,1.0,1.898608,0.98,0.81,4.8,0.052632
2,2024-01-11 00:00:00,false,12,0.281,0.1,0.108333,0.026253,0.7055,0.0,0.317658,0.96,0.85,4.7,0.172424


## Посмотрим на пропуски

In [4]:
df_train.null_count()

query_id,report_date,target,rn,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,3266,0,0,7605609,0,0,0


In [5]:
# Преобразуем DataFrame из Polars в Pandas
df_train_pd = df_train.to_pandas()
df_test_pd = df_test.to_pandas()

# Группируем данные по query_id с сохранением уникальных значений rn
grouped = df_train_pd.groupby("query_id", as_index=False).first()

# Разделение на тренировочный и валидационный наборы
train_query_ids, val_query_ids = train_test_split(
    grouped["query_id"], random_state=42, test_size=0.2)

# Отбор тренировочных и валидационных данных
train_df = df_train_pd[df_train_pd["query_id"].isin(train_query_ids)]
val_df = df_test_pd[df_test_pd["query_id"].isin(val_query_ids)]


У меня не получалось нормально сгруппировать в polars поэтому сделал это в pandas

In [6]:
# Преобразование DataFrame из Pandas в Polars
train_df = pl.from_pandas(train_df)
val_df = pl.from_pandas(val_df)

In [7]:
feature_columns = [f"feature_{i}" for i in range(1, 12)]

## Предобработка признаков

Заполнитель нулей сделал медианными значениями, т.к очень много выбросов в данных

In [8]:
class MedianPLImputer:
    def __init__(self):
        self.feature_medians = {}
        
    def fit(self, x: pl.DataFrame): 
        self.feature_medians = x.median().to_dicts()[0]
        return self
    
    def transform(self, x: pl.DataFrame) -> pl.DataFrame:
        return (
            x
            .with_columns(*[
                pl.col(col).fill_null(val)
                for col, val in self.feature_medians.items()
            ])
        )
    
imputer = MedianPLImputer().fit(train_df.select(*feature_columns))

train_df = imputer.transform(train_df)
val_df = imputer.transform(val_df)

df_test = imputer.transform(df_test)
df_train = imputer.transform(df_train)

## Метрики

Опираться в выборе метрик буду на статью с хабра <br>
https://habr.com/ru/companies/econtenta/articles/303458/

Также возьму метрики из бейзлайна

In [9]:

# Эта функция считает все популярные метрики, которые нашёл. В дальнейшем подумаю какие наиболее подходящие
from scipy.stats import bootstrap


def ap_at_k(relevances, k=10):
    total_relevant = sum(relevances[:k])
    
    if total_relevant == 0:
        return 0
    
    ap_ = 0
    for k_ in range(1, k+1):
        ap_ += sum(relevances[:k_]) * relevances[k_ - 1] / k_
        
    return ap_ / total_relevant
def dcg_at_k(relevances, k=10):
    relevances = np.asarray(relevances)[:k]
    discounts = np.log2(np.arange(len(relevances)) + 2)
    return np.sum(relevances / discounts)

def ndcg_at_k(relevances, k=10):
    ideal_dcg = dcg_at_k(sorted(relevances, reverse=True), k)
    if ideal_dcg == 0:
        return 0
    return dcg_at_k(relevances, k) / ideal_dcg

def precision_at_k(relevances, k=10):
    return np.mean(relevances[:k])

def err_at_k(relevances, k=10):
    p = 1.0
    err = 0.0
    for i in range(min(k, len(relevances))):
        rel = relevances[i]
        err += (2**rel - 1) / (i + 1) * p
        p *= (1 - rel)
    return err

def p_found_at_k(relevances, k=10):
    p = 1.0
    p_found = 0.0
    for i in range(min(k, len(relevances))):
        rel = relevances[i]
        p_found += rel * p
        p *= (1 - rel)
    return p_found / k

def calc_metrics(df, rn_col):
    return (
        df
        .sort_values(rn_col)
        .groupby("query_id")
        .agg(
            map_at_1=("target", lambda x: ap_at_k(x.tolist(), k=1)),
            map_at_5=("target", lambda x: ap_at_k(x.tolist(), k=5)),
            map_at_10=("target", lambda x: ap_at_k(x.tolist(), k=10)),
            recall_at_1=("target", lambda x: sum(x[:1]) / sum(x)),
            recall_at_5=("target", lambda x: sum(x[:5]) / sum(x)),
            recall_at_10=("target", lambda x: sum(x[:10]) / sum(x)),
            mrr=("target", lambda x: 1 / (x.tolist().index(True) + 1)),
            hit_at_1=("target", lambda x: x[:1].sum() > 0),
            hit_at_5=("target", lambda x: x[:5].sum() > 0),
            hit_at_10=("target", lambda x: x[:10].sum() > 0),
            precision_at_1=("target", lambda x: precision_at_k(x.tolist(), k=1)),
            precision_at_5=("target", lambda x: precision_at_k(x.tolist(), k=5)),
            precision_at_10=("target", lambda x: precision_at_k(x.tolist(), k=10)),
            avg_precision_at_1=("target", lambda x: ap_at_k(x.tolist(), k=1)),
            avg_precision_at_5=("target", lambda x: ap_at_k(x.tolist(), k=5)),
            avg_precision_at_10=("target", lambda x: ap_at_k(x.tolist(), k=10)),
            ndcg_at_1=("target", lambda x: ndcg_at_k(x.tolist(), k=1)),
            ndcg_at_5=("target", lambda x: ndcg_at_k(x.tolist(), k=5)),
            ndcg_at_10=("target", lambda x: ndcg_at_k(x.tolist(), k=10)),
            err_at_10=("target", lambda x: err_at_k(x.tolist(), k=10)),
            p_found_at_10=("target", lambda x: p_found_at_k(x.tolist(), k=10))
        )
    )


In [10]:
calc_metrics(df_train.to_pandas(), "rn").agg("mean")

map_at_1               0.141018
map_at_5               0.224893
map_at_10              0.243538
recall_at_1            0.135303
recall_at_5            0.367538
recall_at_10           0.512123
mrr                    0.264451
hit_at_1               0.141018
hit_at_5               0.380652
hit_at_10              0.527629
precision_at_1         0.141018
precision_at_5         0.077806
precision_at_10        0.054622
avg_precision_at_1     0.141018
avg_precision_at_5     0.224893
avg_precision_at_10    0.243538
ndcg_at_1              0.141018
ndcg_at_5              0.257128
ndcg_at_10             0.304591
err_at_10              0.244825
p_found_at_10          0.052763
dtype: float64

In [11]:
calc_metrics(df_test.to_pandas(), "rn").agg("mean")

map_at_1               0.143671
map_at_5               0.228271
map_at_10              0.245740
recall_at_1            0.137466
recall_at_5            0.371391
recall_at_10           0.510242
mrr                    0.267064
hit_at_1               0.143671
hit_at_5               0.385122
hit_at_10              0.524978
precision_at_1         0.143671
precision_at_5         0.078640
precision_at_10        0.054406
avg_precision_at_1     0.143671
avg_precision_at_5     0.228271
avg_precision_at_10    0.245740
ndcg_at_1              0.143671
ndcg_at_5              0.260352
ndcg_at_10             0.305938
err_at_10              0.247351
p_found_at_10          0.052498
dtype: float64

# Модель

Построил логистическую регрессию на парах (релевантный - нерелевантный отклик) с учётом позиционного смещения 

In [ ]:
from tqdm import tqdm
from joblib import Parallel, delayed
from itertools import chain
import numpy as np
import polars as pl

def batchify(sequence, batch_size, bar=False, leave=True):
    if batch_size is None:
        yield sequence
    else:
        l = len(sequence)

        for ndx in tqdm(range(0, l, batch_size), disable=not bar, total=l // batch_size + 1, leave=leave):
            yield sequence.slice(ndx, min(ndx + batch_size, l))


def mine_pairs(df_group, random_gen, n_neg_per_pos=1):
    positive_df = df_group.filter(pl.col("target") == True)
    n_positive_nmids = positive_df.shape[0]

    negative_df = df_group.filter(pl.col("target") == False)
    n_negative = min(n_neg_per_pos * n_positive_nmids, negative_df.shape[0])

    ids = random_gen.choice(range(negative_df.shape[0]), size=n_negative, replace=False)
    negative_dfs = list(batchify(negative_df, batch_size=n_positive_nmids, bar=False))

    return {"positive": positive_df, "negatives": negative_dfs, "rn": df_group["rn"]}


def make_substact_features(mined_dfs, feature_columns):
    return [
        (mined_dfs["positive"].select(feature_columns + ["rn"]) - df.select(feature_columns + ["rn"])).to_numpy()
        for df in mined_dfs["negatives"]
    ]

def _f(query_id, df_group, feature_columns, random_state):
    random_gen = np.random.default_rng(random_state)

    mined_dfs = mine_pairs(df_group, random_gen, n_neg_per_pos=1)
    features_ = make_substact_features(mined_dfs, feature_columns)
    ids_ = [[query_id]] * sum(f.shape[0] for f in features_)
    rn_ = [df["rn"].to_numpy() for df in mined_dfs["negatives"]]  

    return features_, ids_, rn_  # Возвращаем также значение rn


random_gen = np.random.default_rng(342)
results = Parallel(n_jobs=10)(
    delayed(_f)(query_id, df_group, feature_columns, random_gen.integers(100000, size=1)[0])
    for query_id, df_group in tqdm(df_train.groupby("query_id"), total=df_train.select("query_id").n_unique()))

features = list(chain.from_iterable([f for f, _, _ in results]))
ids = list(chain.from_iterable([i for _, i, _ in results]))
rn = list(chain.from_iterable([r for _, _, r in results]))

rn_flat = [item for sublist in rn for item in sublist]  
query_ids_flat = [query_id[0] for sublist in ids for query_id in sublist] 
features_df = pl.DataFrame({
    f"feature_{i+1}": np.concatenate([f[:, i] for f in features]) for i in range(len(feature_columns))
})
features_df = features_df.with_columns(target=pl.Series([1] * features_df.shape[0]))
features_df = features_df.with_columns(
    rn=pl.Series(rn_flat, dtype=pl.Int64),  
    query_id=pl.Series(query_ids_flat, dtype=pl.UInt32)
)


В этом методе переписал часть функций для возможного ускорения работы

Очень долго и безуспешно пытался просто поделить на батчи и сгладить изначальные данные, но не получилось. 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline


def get_model(degree: int = 1, include_bias=False, **logreg_params):
    return make_pipeline(
        PolynomialFeatures(degree=degree, include_bias=include_bias),
        LogisticRegression(
            random_state=42, 
            max_iter=1000,
            solver="liblinear",
            **logreg_params
        )
    )

train_query_ids_set = set(train_query_ids.to_list())

train_df = features_df.filter(pl.col("query_id").apply(lambda x: x in train_query_ids_set))

model = get_model(degree=1, penalty="l1", C=0.1)
model.fit(
    train_df[feature_columns], 
    train_df["target"],
    logisticregression__sample_weight=train_df["rn"]  # Использование rn в качестве весовых коэффициентов
)


В дальнейшем попробую lgbm с алгоритмом для устранения позиционного отклонения. <br>
В статье https://arxiv.org/pdf/1809.05818.pdf боролись с подобной проблемой и модель с Pairwise Debiasing получилось добиться наилучших результатов.

Также попробую NN модели. В статье https://arxiv.org/abs/1810.09591 исследовали ранжирование в поиске и показали, что применение нейронок может быть очень полезно